### Chat Message

In [ ]:
from langchain.chains import LLMChain
from langchain.memory import ConversationBufferMemory
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_google_genai.embeddings import GoogleGenerativeAIEmbeddings
from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
import google.generativeai as genai
import os
from dotenv import load_dotenv

In [ ]:
host = os.getenv("HOST")
port = os.getenv("PORT")
genai.configure(api_key = os.getenv("GOOGLE_API_KEY"))

In [ ]:
template = """You are having a Conversation with a human.
{chat_history}
Human: {human_input}
chatbot:
"""
prompt = PromptTemplate(template= template, input_variables=["chat_history", "human_input"])
memory = ConversationBufferMemory(memory_key="chat_history")

In [ ]:
llm = ChatGoogleGenerativeAI(model = "gemini-pro")
output_parser = StrOutputParser()
embeddings = GoogleGenerativeAIEmbeddings(model = "models/embedding-001")


In [ ]:
from langchain.memory import ChatMessageHistory
history = ChatMessageHistory()
history.add_user_message("Hi")
history.add_ai_message("Hi! How can i help you?")

In [ ]:
history.messages

### Using ConversationBufferMemory

In [ ]:
from langchain.chains import LLMChain
from langchain.memory import ConversationBufferMemory
from langchain_core.prompts import PromptTemplate
from langchain_core.prompts import ChatPromptTemplate

In [ ]:
from langchain.llms.ollama import Ollama

In [ ]:
from langchain.llms.ollama import Ollama
llm = Ollama(model="llama2")    

In [ ]:
template = """ You are a chatbot who is having a conversation with a human.
{chat_history}
Human: {human_input}
Bot:
"""
prompt = PromptTemplate(
    input_variables=["chat_history", "human_input"], template=template
)
memory = ConversationBufferMemory(memory_key="chat_history")

llmChain = LLMChain(
    llm = llm,
    prompt=prompt, 
    memory= memory,
    verbose=True
)

In [ ]:
llmChain.run("hi")

In [ ]:
llmChain.predict(human_input = "I am fine. How are you?")

In [ ]:
from langchain.prompts import ( ChatPromptTemplate, 
                                HumanMessagePromptTemplate,
                                MessagesPlaceholder)
from langchain_core.messages import SystemMessage

In [ ]:
prompt = ChatPromptTemplate.from_messages(
    [
        SystemMessage(
            content = "You are a chatbot having a conversation with human."
        ),
        MessagesPlaceholder  (
            variable_name = "chat_history"
        ),
        HumanMessagePromptTemplate.from_template(
            "{human_input}"
        )
]
)
memory = ConversationBufferMemory(memory_key="chat_history",return_messages=True)

In [ ]:
llm = Ollama(model = "llama2")
chat_llm_chain = LLMChain(
    llm = llm,
    memory= memory,
    prompt=prompt, 
    verbose=True
)

In [ ]:
chat_llm_chain.predict(human_input="Hi there my friend")

In [ ]:
from langchain.prompts import(HumanMessagePromptTemplate, MessagesPlaceholder, ChatPromptTemplate)
from langchain_core.messages import SystemMessage
from langchain.llms.ollama import Ollama
from langchain.memory import ConversationBufferMemory
from langchain.chains import LLMChain

llm = Ollama(model = "llama2")
prompt = ChatPromptTemplate.from_messages(
    [
        SystemMessage(content="You are a chatbot having a conversation with a human"),
        MessagesPlaceholder(variable_name="chat_history"),
        HumanMessagePromptTemplate.from_template("{human_input}")
    ]
)
memory = ConversationBufferMemory(memory_key= "chat_history", return_messages=True)
llm_chain = LLMChain(
    llm = llm, 
    memory = memory, 
    prompt = prompt,
    verbose=True
)
llm_chain.predict(human_input = "Hi there")


### Up until now the llm is only able to take single input. So to build a model with a capability to have multiple QnA. we do the following:
1. Read the file to give it to the model to read the content.
2. Split the texts
3. Make embedings
4. Similarity search from the database
5. Template, Memory, Chain

In [ ]:
from langchain_community.vectorstores import Chroma
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.chains.question_answering import load_qa_chain
from langchain.memory import ConversationBufferMemory
from langchain_core.prompts import PromptTemplate
import google.generativeai as genai
from langchain.llms.ollama import Ollama

In [ ]:
# Reading the file
with open("state_of_the_union.txt") as f:
    state_of_the_union = f.read()
state_of_the_union

In [ ]:
# Splitting the text
text_splitter = CharacterTextSplitter(chunk_size = 1000, chunk_overlap = 200)
texts = text_splitter.split_text(state_of_the_union)

In [ ]:
# Embedding models 
embeddings = GoogleGenerativeAIEmbeddings(model = "models/embedding-001")

In [ ]:
docsearch = Chroma.from_texts(texts = texts,
                              embedding= embeddings,
                              metadatas = [{"source": i} for i in range(len(texts))]
                              )

In [ ]:
query = "What did the president say about Justice Breyer"
docs = docsearch.similarity_search(query)

In [ ]:
docs

In [ ]:
template = """ 

You are a chatbot who is having a conversation with a human.
Given the following extracted parts of a long document and a question, create a final answer.
{context}
{chat_history}
Human: {human_input}
Chatbot:"""
llm = Ollama(model = "llama2")

prompt = PromptTemplate(input_variables=["context", "chat_history", "human_input"], template= template)
memory = ConversationBufferMemory(memory_key= "chat_history", input_key= "human_input")
chain = load_qa_chain(memory= memory, prompt = prompt, chain_type= "stuff",llm =llm )


In [ ]:
query = "What did the president say about Justice Bryer"
chain({"input_documents": docs, "human_input": query}, return_only_outputs=True)

In [ ]:
print(chain.memory.buffer)

To continue further we need to learn more about Agents so now goto ../Agents/ to 

In [ ]:
from langchain.vectorstores import FAISS

# Conversational Form

In [38]:
from langchain.llms.ollama import Ollama
from langchain.chains import TransformChain, LLMChain, SimpleSequentialChain
from langchain.vectorstores import FAISS
import google.generativeai as genai
from langchain.chat_models import ChatOpenAI
from pydantic import BaseModel, Field, EmailStr
from langchain.schema.output_parser import StrOutputParser
from langchain.prompts import ChatPromptTemplate, PromptTemplate
from langchain_google_genai.embeddings import GoogleGenerativeAIEmbeddings
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.chains import create_tagging_chain,create_tagging_chain_pydantic
import os
from dotenv import load_dotenv
from pydantic import BaseModel, Field

load_dotenv()

# model = Ollama(model = "llama2")
llm = ChatOpenAI(model='gpt-3.5-turbo-0613', temperature=0)
# model = ChatGoogleGenerativeAI(model = 'gemini-pro', temperature=0)
embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")

template = """
### Instructions
You are a virtual assistant for a company named Gojo and co. You are given 
context in which you should answer the queries from which is given between two back ticks.\
You are given queried by the user in triple backticks \
the user query with relevant answers and do not provide irrevant answers, STRICTLY.\
ANSWER WITHIN 50 WORDS.
query: ```{query}```
context: ``{context}``
Provide the answers without backticks.
Ask the user their Name, Email and PhoneNumber at the very beginning.
"""
prompt = ChatPromptTemplate.from_template(template=template)
# database = FAISS.load_local("ConversationalFormDemo",embeddings)

class PersonalDetails(BaseModel):
    first_name: str = Field(
        ...,
        description="This is the first name of the user.",
    )
    last_name: str = Field(
        ...,
        description="This is the last name of the user.",
    )
    email: str = Field(
        ...,
        description="This is the email of the user",
    )

chain = create_tagging_chain_pydantic(PersonalDetails,llm)

# def get_user_input() -> PersonalDetails:
#     first_name = input("Enter your first name: ")
#     last_name = input("Enter your last name: ")
#     age = input("Enter your Age: ")
#     return PersonalDetails(first_name=first_name,last_name = last_name, age=age)


In [39]:
# test_string = "My last name is acharya and you can contact me at ajeetach@gmail.com."
test_string_1 = "my first name is Ajeet"
res = chain.run(test_string_1)
res

PersonalDetails(first_name='Ajeet', last_name='', email='')

In [40]:
user_123_personal_details = PersonalDetails(first_name="",
                                last_name="",
                                email="",
)
user_123_personal_details

PersonalDetails(first_name='', last_name='', email='')

In [41]:
def check_what_is_empty(user_personal_details):
    ask_for = []
    for field, value in user_personal_details.dict().items():
        if value in [None, "", 0]:
            print(f"Field '{field} is empty.'")
            ask_for.append(f'{field}')
    return ask_for

In [42]:
ask_for = check_what_is_empty(user_123_personal_details)
ask_for

Field 'first_name is empty.'
Field 'last_name is empty.'
Field 'email is empty.'


['first_name', 'last_name', 'email']

In [43]:
## checking the response and adding it
def add_non_empty_details(current_details: PersonalDetails, new_details: PersonalDetails):
    non_empty_details = {k: v for k, v in new_details.dict().items() if v not in [None, ""]}
    updated_details = current_details.copy(update=non_empty_details)
    return updated_details

In [44]:
user_123_personal_details = add_non_empty_details(user_123_personal_details, res)
user_123_personal_details

PersonalDetails(first_name='Ajeet', last_name='', email='')

In [45]:
res = chain.run(test_string)
user_123_personal_details = add_non_empty_details(user_123_personal_details, res)
user_123_personal_details

PersonalDetails(first_name='Ajeet', last_name='acharya', email='ajeetach@gmail.com')

In [46]:
ask_for = check_what_is_empty(user_123_personal_details)

In [47]:
# if ask_for does not have Empty strings
if not ask_for:
    print("Thank you wee have all the details")

Thank you wee have all the details


### Putting it in LLMCHAIN

In [48]:
user_details = PersonalDetails(first_name="",
                                last_name="",
                                email="",
)
user_details

PersonalDetails(first_name='', last_name='', email='')

In [49]:
def ask_for_info(ask_for = ['first_name','last_name', "email"]):
    first_prompt = ChatPromptTemplate.from_template(
        "Below is are some things to ask the user for in a coversation way. \
        you should only ask one question at a time even if you don't get all the info \
        don't ask as a list! Don't greet the user! Don't say Hi.\
        Explain you need to get some info.\
        If the ask_for list is empty then thank \
        them and ask how you can help them \n\n \
        ### ask_for list: {ask_for}"
    )
    
    info_chain = LLMChain(llm = llm, prompt=first_prompt)
    ai_chat= info_chain.run(ask_for = ask_for)
    return ai_chat

In [50]:
def filter_response(text_input, user_details):
    chain = create_tagging_chain_pydantic(PersonalDetails, llm)
    res = chain.run(text_input)
    user_details = add_non_empty_details(user_details, res)
    ask_for = check_what_is_empty(user_details)
    return user_details, ask_for

In [51]:
ask_for_info()

'Can I please have your first name?'

In [52]:
# text_input = "Ok. My name is Ajeet"
user_details, ask_for = filter_response(text_input, user_details)
user_details

PersonalDetails(first_name='John', last_name='Doe', email='johndoe@example.com')

In [ ]:
if ask_for:
    ai_response = ask_for_info(ask_for)
    print(ai_response)
else:
    print("Everything gethered move to next phase")

In [ ]:
# text_input = "My name is Ajeet acharya"
user_details, ask_for = filter_response(text_input, user_details)

In [ ]:
if ask_for:
    ai_response = ask_for_info(ask_for)
    print(ai_response)
else:
    print("Everything gethered move to next phase")

In [ ]:
# text_input = "Contact me at Ajeet@gmail.com"
user_details, ask_for = filter_response(text_input, user_details)

In [ ]:
user_details

In [ ]:
import redis
REDIS_SERVER = os.getenv('REDIS_SERVER') or "localhost"
cache = redis.Redis(REDIS_SERVER)

# Test

In [53]:
from langchain.llms.ollama import Ollama
from langchain.chains import TransformChain, LLMChain, SimpleSequentialChain
from langchain.vectorstores import FAISS
import google.generativeai as genai
from langchain.chat_models import ChatOpenAI
from pydantic import BaseModel, Field, EmailStr
from langchain.schema.output_parser import StrOutputParser
from langchain.prompts import ChatPromptTemplate, PromptTemplate
from langchain_google_genai.embeddings import GoogleGenerativeAIEmbeddings
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.chains import create_tagging_chain,create_tagging_chain_pydantic
import os
from dotenv import load_dotenv
from pydantic import BaseModel, Field

load_dotenv()

# model = Ollama(model = "llama2")
llm = ChatOpenAI(model='gpt-3.5-turbo-0613', temperature=0)
# model = ChatGoogleGenerativeAI(model = 'gemini-pro', temperature=0)
embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")


# prompt = ChatPromptTemplate.from_template(template=template)
# database = FAISS.load_local("ConversationalFormDemo",embeddings)

class PersonalDetails(BaseModel):
    first_name: str = Field(
        ...,
        description="This is the first name of the user.",
    )
    last_name: str = Field(
        ...,
        description="This is the last name of the user.",
    )
    email: str = Field(
        ...,
        description="This is the email of the user",
    )

chain = create_tagging_chain_pydantic(PersonalDetails,llm)

# test_string = "My last name is acharya and you can contact me at ajeetach@gmail.com."
# test_string_1 = "my first name is Ajeet"
# print(chain.run(test_string_1))

# def get_user_input() -> PersonalDetails:
#     first_name = input("Enter your first name: ")
#     last_name = input("Enter your last name: ")
#     age = input("Enter your Age: ")
#     return PersonalDetails(first_name=first_name,last_name = last_name, age=age)

In [81]:
def check_what_is_empty(user_personal_details):
    ask_for = []
    for field, value in user_personal_details.dict().items():
        if value in [None, "", 0]:
            print(f"Field '{field} is empty.'")
            ask_for.append(f'{field}')
    return ask_for


## checking the response and adding it
def add_non_empty_details(current_details: PersonalDetails, new_details: PersonalDetails):
    non_empty_details = {k: v for k, v in new_details.dict().items() if v not in [None, ""]}
    updated_details = current_details.copy(update=non_empty_details)
    return updated_details


def ask_for_info(ask_for):
    first_prompt = ChatPromptTemplate.from_template(
        "Below is are some things to ask the user for in a coversation way. \
        you should only ask one question at a time even if you don't get all the info \
        don't ask as a list! Don't greet the user! Don't say Hi.\
        Explain you need to get some info.\
        If the ask_for list is empty then thank \
        them and ask how you can help them. Do not fill garbage values to the fields. If the user does not enter details do not fill it with \
            answers like\n\n \
        ### ask_for list: {ask_for}"
    )
    
    info_chain = LLMChain(llm = llm, prompt=first_prompt)
    ai_chat= info_chain.run(ask_for = ask_for)
    return ai_chat

def filter_response(text_input, user_details):
    chain = create_tagging_chain_pydantic(PersonalDetails, llm)
    res = chain.run(text_input)
    user_details = add_non_empty_details(user_details, res)
    ask_for = check_what_is_empty(user_details)
    
    return user_details, ask_for


def filter_response(text_input, user_details):
    chain = create_tagging_chain_pydantic(PersonalDetails, llm)
    res = chain.run(text_input)
    user_details = add_non_empty_details(user_details, res)
    ask_for = check_what_is_empty(user_details)
    print(user_details)
    return user_details, ask_for

In [74]:
user_details = PersonalDetails(first_name="",
                                last_name="",
                                email="",
)
user_details

PersonalDetails(first_name='', last_name='', email='')

In [67]:
ask_for_info()

'Can I please get your first name?'

In [83]:

ques = ask_for_info()
ask_for=  ['first_name','last_name', "email"]
text_input =input(ques)
user_details, ask_for = filter_response(text_input, user_details)
print(ask_for)

ValidationError: 1 validation error for PersonalDetails
email
  field required (type=value_error.missing)

In [79]:
ques = ask_for_info() 
text_input =input(ques)
user_details, ask_for = filter_response(text_input, user_details)


Field 'last_name is empty.'
Field 'email is empty.'
first_name='Ajeet' last_name='' email=''


In [89]:
user_details = PersonalDetails(first_name="",
                                last_name="",
                                email="",
)
user_details
ask_for=  ['first_name','last_name', "email"]

In [91]:
# Define your functions and classes here

# Initialize user_details




Field 'email is empty.'
first_name='ajeet' last_name='acharya' email=''
['email']
first_name='ajeet' last_name='acharya' email='ajeet@gmail.com'
[]


In [ ]:
ask_for_info()
text_input = "Ok. My name is Ajeet"
user_details, ask_for = filter_response(text_input, user_details)
if ask_for:
    ai_response = ask_for_info(ask_for)
    print(ai_response)
else:
    print("Everything gethered move to next phase")
    print(user_details)
    
text_input = "Ok. My name is Ajeet Acharya and please contact me at ajeet@gmail.com"
print(text_input)
user_details, ask_for = filter_response(text_input, user_details)
if ask_for:
    ai_response = ask_for_info(ask_for)
    print(ai_response)
else:
    print("Everything gethered move to next phase")
    
text_input = "Ok. My name is Ajeet Acharya and please contact me at ajeet@gmail.com"
print(text_input)
user_details, ask_for = filter_response(text_input, user_details)


In [ ]:
ask_for_info()
text_input ="Ajeet Acharya is my name"
# user_details, ask_for = filter_response(text_input, user_details)
# if ask_for:
#     ai_response = ask_for_info(ask_for)
#     input
#     print(ai_response)
# else:
#     print("Everything gathered move to next phase")
# print(user_details)

In [ ]:
ques = ask_for_info()

In [ ]:
text_input = input(ques)
user_details, ask_for = filter_response(text_input, user_details)
user_details

In [ ]:
ask_for_info()
text_input = "Ok. My name is Ajeet"
user_details, ask_for = filter_response(text_input, user_123_personal_details)
if ask_for:
    ai_response = ask_for_info(ask_for)
    print(ai_response)
else:
    print("Everything gethered move to next phase")
text_input = "Ok. My name is Ajeet Acharya and please contact me at ajeet@gmail.com"
user_details, ask_for = filter_response(text_input, user_details)
if ask_for:
    ai_response = ask_for_info(ask_for)
    print(ai_response)
else:
    print("Everything gethered move to next phase")
text_input = "Ok. My name is Ajeet Acharya and please contact me at ajeet@gmail.com"
user_details, ask_for = filter_response(text_input, user_details)
user_details

In [ ]:
text_input = "Ok. My name is Ajeet Acharya and please contact me at ajeet@gmail.com"
user_details, ask_for = filter_response(text_input, user_details)
user_details